# VacationPy
----

Planning future vacations

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Importing API key
from api_keys import g_key

### Store Part I results into DataFrame
* Loading the csv exported in Part I to a DataFrame

In [2]:
#Reading and opening Weatherpy DataFrame
data_file = os.path.join("output_data","cities.csv")
weather_df= pd.read_csv(data_file)
weather_df.head()


,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Albany,US,42.60,-73.97,63.00,57,21,6.06,1592183161
1,Ushuaia,AR,-54.80,-68.30,33.80,86,75,6.87,1592183161
2,Yangi Marg`ilon,UZ,40.43,71.72,75.20,29,92,4.70,1592183161
3,São Filipe,CV,14.90,-24.50,76.59,79,0,6.51,1592183161
4,Manta,EC,-0.95,-80.73,73.40,88,90,8.05,1592183004


### Humidity Heatmap

In [3]:
# Accessing maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
# Storing latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]

# Storing humidity in a variable that will be used as the "Weight" parameter for the heat map
humidity_values = weather_df["Humidity"].astype(float)

# Plotting Heatmap
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Defining maps parameters
fig = gmaps.figure(zoom_level=1.4, center= (51.4934,0.0098), layout=figure_layout)

# Creating heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_values, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Adding the  layer
fig.add_layer(heat_layer)

# Displaying the  figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Filtering and adjusting the source dataframe to find desired weather conditions for future vacations

In [5]:
# Filtering ideal weather conditions, and creting a new dataframe
hotel_df= weather_df.loc[((weather_df["Max Temp"]>80) & (weather_df["Max Temp"]<=90)) & (weather_df["Wind Speed"]<10) \
                              & (weather_df["Humidity"]<50) & (weather_df["Cloudiness"]<5)].copy()
                                                                                                     
hotel_df

# Null values were removed in the previous analysis.

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
295,Meybod,IR,32.25,54.02,80.60,10,0,4.70,1592183211
338,Fort Smith,US,35.39,-94.40,89.60,43,1,9.17,1592183123
372,Bilma,NE,18.69,12.92,88.97,12,0,5.50,1592183223
512,Gazli,UZ,40.13,63.45,81.86,15,0,5.88,1592183244
529,Riyadh,SA,24.69,46.72,84.20,28,0,3.09,1592183212


### Hotel Map

In [6]:
# Adding empty columns to hotel_df
hotel_df["Hotel Name"]= ""
hotel_df["Lat"]= ""
hotel_df["Lng"]= ""
hotel_df

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name,Lat,Lng
295,Meybod,IR,32.25,54.02,80.60,10,0,4.70,1592183211,,,
338,Fort Smith,US,35.39,-94.40,89.60,43,1,9.17,1592183123,,,
372,Bilma,NE,18.69,12.92,88.97,12,0,5.50,1592183223,,,
512,Gazli,UZ,40.13,63.45,81.86,15,0,5.88,1592183244,,,
529,Riyadh,SA,24.69,46.72,84.20,28,0,3.09,1592183212,,,


In [7]:
# Defining API url parameters
params = {
    "radius": 5000,
    "type":"lodging",
    "keyword":"hotel",
    "key": g_key
}

# Looping through each dataframe index and rows
for index, row in hotel_df.iterrows():
    # getting lat, lng from hotel_df
    lat = row["Latitude"]
    lng = row["Longitude"]
# changing location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Defining base Google Place API url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # making the request and getting the json response
    hotel_response = requests.get(base_url, params=params).json()
# Creating an exception to avoid keyerrors    
    try:
    
        print(f"Nice! we have found {hotel_response['results'][0]['name']} in {row['City']}")
        
        hotel_df.loc[index,'Hotel Name'] = hotel_response['results'][0]['name']
        hotel_df.loc[index,'Lat'] = hotel_response['results'][0]['geometry']['location']['lat']
        hotel_df.loc[index,'Lng'] = hotel_response['results'][0]['geometry']['location']['lng']
        
    except:
        #Printing an explanation message
        print(f"No Hotel Found in {row['City']}")
        # For those hotels that are not founded, th program must put np.NaN values over those cells
        hotel_df.loc[index,'Hotel Name'] = np.NaN
        hotel_df.loc[index,'Lat']= np.NaN
        hotel_df.loc[index,'Lng'] = np.NaN
        pass
    
# Dropping np.NaN values    
hotel_df.dropna(inplace=True)
        
    
# Organizing and printing hotel_df 

hotel_df= hotel_df[["City","Country","Lat","Lng","Hotel Name"]]
hotel_df

Nice! we have found هتل ناصر in Meybod
Nice! we have found DoubleTree by Hilton Fort Smith City Center in Fort Smith
No Hotel Found in Bilma
No Hotel Found in Gazli
Nice! we have found Four Seasons Hotel Riyadh At Kingdom Center in Riyadh


,City,Country,Lat,Lng,Hotel Name
295,Meybod,IR,32.226,54.024,هتل ناصر
338,Fort Smith,US,35.3854,-94.4253,DoubleTree by Hilton Fort Smith City Center
529,Riyadh,SA,24.7115,46.6745,Four Seasons Hotel Riyadh At Kingdom Center


In [8]:
# Defining info_box_template
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Storing info_box_content in hotel_info variable
# A list comprehension loop to store hotel info values
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Adding marker layer ontop of heat map
markers= gmaps.marker_layer(locations,info_box_content= hotel_info)
fig.add_layer(markers)

# Displaying figure

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…